In [9]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 15393, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 15393 (delta 9), reused 16 (delta 6), pack-reused 15369
Receiving objects: 100% (15393/15393), 14.37 MiB | 33.15 MiB/s, done.
Resolving deltas: 100% (10519/10519), done.


In [8]:
import cv2
import pytesseract
import torch
import numpy as np
from utils.general import non_max_suppression

# Load the YOLOv5 object detection model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Load the image
img = cv2.imread('topol.jpg')

# Run the object detection model on the image
results = model(img)

# Filter the results to only include license plates
# plates = results.pred[results.pred[:, -1] == 0]

# # Apply non-maximum suppression to remove overlapping plates
# plates = non_max_suppression(plates, conf_thres=0.5, iou_thres=0.5)

# # Loop over the detected license plates
# for plate in plates:
#     # Extract the coordinates of the license plate
#     x1, y1, x2, y2 = plate[:4].detach().numpy()
    
#     # Crop the license plate from the image
#     plate_img = img[int(y1):int(y2), int(x1):int(x2)]
    
#     # Convert the license plate image to grayscale
#     plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    
#     # Apply thresholding to the license plate image
#     thresh = cv2.threshold(plate_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
#     # Extract the text from the license plate image using OCR
#     plate_no = pytesseract.image_to_string(thresh, config='--psm 11')
    
#     # Draw a rectangle around the license plate in the original image
#     cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
#     # Draw the license plate number on the original image
#     if plate_no:
#         cv2.putText(img, plate_no.strip(), (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# # Display the original image with the license plate number
# cv2.imshow('License Plate Detection', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'utils'